In [1]:
val spark2 = spark
import spark2.implicits._

In [2]:
def creatStream(port: Int, duration: Int) {
    val lines = (spark.readStream
                      .format("socket")
                      .option("host", "localhost")
                      .option("port", port)
                      .load())
    
    val counts = (lines.as[String]
                       .map(line => line.replaceAll("[^a-zA-Z0-9\\s]", ""))
                       .flatMap(line => line.split("\\s+"))
                       .filter(_.nonEmpty)
                       .groupByKey(_.toLowerCase)
                       .count()
                       .orderBy($"count(1)" desc))
    
    val query = (counts.orderBy($"count(1)" desc)
                       .writeStream
                       .outputMode("complete")
                       .format("console")
                       .start
                       .awaitTermination(duration))
}

In [3]:
// nc -lk 12345 < war_and_peace.txt

creatStream(12345, 10000)

-------------------------------------------
Batch: 0
-------------------------------------------
+-------+--------+
|  value|count(1)|
+-------+--------+
|    the|    2826|
|    and|    1861|
|     to|    1456|
|     of|    1035|
|      a|    1012|
|     he|     772|
|    his|     757|
|     in|     754|
|    her|     675|
|      i|     591|
|    was|     575|
|    you|     572|
|   that|     572|
|   with|     541|
|     it|     463|
|    she|     459|
|     at|     404|
|chapter|     395|
|     is|     394|
|     as|     380|
+-------+--------+
only showing top 20 rows

-------------------------------------------
Batch: 1
-------------------------------------------
+-----+--------+
|value|count(1)|
+-----+--------+
|  the|   34570|
|  and|   22159|
|   to|   16716|
|   of|   14991|
|    a|   10521|
|   he|    9809|
|   in|    8801|
|  his|    7967|
| that|    7813|
|  was|    7329|
| with|    5695|
|  had|    5354|
|   it|    5179|
|  her|    4700|
|  not|    4658|
|  him|    4574|
|

In [4]:
// nc -lk 12345, then type strings in the console

creatStream(12345, 10000)

-------------------------------------------
Batch: 0
-------------------------------------------
+-----+--------+
|value|count(1)|
+-----+--------+
|hello|       1|
+-----+--------+

-------------------------------------------
Batch: 1
-------------------------------------------
+-----+--------+
|value|count(1)|
+-----+--------+
|hello|       2|
|world|       1|
+-----+--------+

-------------------------------------------
Batch: 2
-------------------------------------------
+-----+--------+
|value|count(1)|
+-----+--------+
|hello|       4|
|world|       2|
+-----+--------+

-------------------------------------------
Batch: 3
-------------------------------------------
+------+--------+
| value|count(1)|
+------+--------+
| hello|       5|
| world|       3|
|yuqing|       1|
+------+--------+

-------------------------------------------
Batch: 4
-------------------------------------------
+------+--------+
| value|count(1)|
+------+--------+
| hello|       6|
| world|       3|
|yuqin

In [5]:
import sys.process._

"more summer.txt" !

::::::::::::::
summer.txt
::::::::::::::
Shall I compare thee to a summer’s day?
Thou art more lovely and more temperate.
Rough winds do shake the darling buds of May,
And summer’s lease hath all too short a date.
Sometime too hot the eye of heaven shines,
And often is his gold complexion dimmed;
And every fair from fair sometime declines,
By chance, or nature’s changing course, untrimmed;
But thy eternal summer shall not fade,
Nor lose possession of that fair thou ow’st,
Nor shall death brag thou wand’rest in his shade,
When in eternal lines to Time thou grow’st.
So long as men can breathe, or eyes can see,
So long lives this, and this gives life to thee.

In [24]:
val port = 12345

// Broadcast file on port one line at a time
(new Thread {
    override def run {
        s"scala Broadcast.scala ${port} summer.txt" !
    }
}).start

In [25]:
creatStream(port, 15000)

-------------------------------------------
Batch: 0
-------------------------------------------
+-------+--------+
|  value|count(1)|
+-------+--------+
|    day|       1|
|   thee|       1|
|      i|       1|
|  shall|       1|
|      a|       1|
|summers|       1|
|     to|       1|
|compare|       1|
+-------+--------+

-------------------------------------------
Batch: 1
-------------------------------------------
+-------+--------+
|  value|count(1)|
+-------+--------+
|   more|       2|
|    too|       2|
|    the|       2|
|    and|       2|
|     of|       2|
|      a|       2|
|summers|       2|
|    art|       1|
|    day|       1|
|   thou|       1|
| lovely|       1|
|    hot|       1|
|  winds|       1|
|   thee|       1|
|   buds|       1|
| shines|       1|
|    eye|       1|
|    may|       1|
|  lease|       1|
|darling|       1|
+-------+--------+
only showing top 20 rows

-------------------------------------------
Batch: 2
------------------------------------------